# Exercises 20-11-2023

## Exercise 8.1
Simulating an experiment with a “Toy Experiment”
Sequences of pseudo-random numbers are often used to simulate the statistical behavior of a measurement experiment or to perform numerical integrations.

According to the frequentist paradigm of statistics, uncertainties of a measurement are derived from its probability density distribution, assuming that the experiment used for that measurement is repeated a large number of times.

Operationally, the experiment leading to the final measurement result is unique, so certain statistical behaviors can only be simulated.

The simulation of a measurement experiment is called a toy experiment.

### Exercise 8.1.1
To determine the precision on the mean of a sample, it’s necessary to generate the sample many times to observe the distribution of mean values: